# **Tarea 1:** EDA y modelos bayesianos
## **Grupo 5** 
## **Integrantes:** 
 * Diego Irarrazaval
 * Pablo Paredes
 * Tomas Rojas

## Pregunta 1:  Carga y limpieza de datos.

In [2]:
import pandas as pd
import glob as glob

In [40]:
files_raw = glob.glob('data/raw/**/*.csv', recursive = True)
files_estadisticas = glob.glob('data/estadisticas_upz/*.csv')

In [41]:
print(files_raw)
print(files_estadisticas)

['data/raw\\w13\\metrocuadrado_all_w13.csv', 'data/raw\\w13\\metrocuadrado_furnished_w13.csv', 'data/raw\\w14\\metrocuadrado_all_w14.csv', 'data/raw\\w14\\metrocuadrado_furnished_w14.csv', 'data/raw\\w15\\metrocuadrado_all_w15.csv', 'data/raw\\w15\\metrocuadrado_furnished_w15.csv', 'data/raw\\w16\\metrocuadrado_all_w16.csv', 'data/raw\\w16\\metrocuadrado_furnished_w16.csv', 'data/raw\\w17\\metrocuadrado_all_w17.csv', 'data/raw\\w17\\metrocuadrado_furnished_w17.csv']
['data/estadisticas_upz\\estadisticas_poblacion.csv', 'data/estadisticas_upz\\indice_inseguridad.csv', 'data/estadisticas_upz\\porcentaje_areas_verdes.csv']
